# PRE-PROCESSING: CATEGORY COLUMN SPLIT

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import glob
import functools
import io

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

In [26]:
pd.set_option('display.max_columns', None)

In [2]:
kick_inter = pd.read_csv('../../data/02_intermediate/kick_deduped.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (14,18,19,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
kick_inter.drop(columns='index', inplace=True)

In [4]:
kick_inter.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'last_update_published_at',
       'launched_at', 'location', 'name', 'permissions', 'pledged', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'unread_messages_count', 'unseen_activity_count',
       'urls', 'usd_pledged', 'usd_type'],
      dtype='object')

**LET'S BREAK UP THE CATEGORY VARIABLE**

In [5]:
kick_inter.category[0]

'{"id":263,"name":"Apparel","slug":"fashion/apparel","position":2,"parent_id":9,"color":16752598,"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/fashion/apparel"}}}'

In [6]:
category = kick_inter['category'].str.partition("name\":\"")[2]
kick_inter['sub_category'] = category.str.partition("\"")[0]

In [7]:
sub_category = kick_inter['category'].str.partition("slug\":\"")[2]
kick_inter['overall_category'] = sub_category.str.partition("\"")[0]

In [8]:
kick_inter.overall_category.nunique()

170

In [9]:
kick_inter.sub_category.nunique()

159

In [22]:
kick_inter.drop(columns='category', inplace=True)

**LET'S BREAK UP THE LOCATION VARIABLE**

In [10]:
kick_inter.location[0]

'{"id":2514971,"name":"Wasilla","slug":"wasilla-ak","short_name":"Wasilla, AK","displayable_name":"Wasilla, AK","localized_name":"Wasilla","country":"US","state":"AK","type":"Town","is_root":false,"urls":{"web":{"discover":"https://www.kickstarter.com/discover/places/wasilla-ak","location":"https://www.kickstarter.com/locations/wasilla-ak"},"api":{"nearby_projects":"https://api.kickstarter.com/v1/discover?signature=1563485120.ec5780d97f7e6574d1bc73fb20e5524556ccc36b&woe_id=2514971"}}}'

In [11]:
kick_inter.location[8975]

'{"id":12723,"name":"Birmingham","slug":"birmingham-gb","short_name":"Birmingham, UK","displayable_name":"Birmingham, UK","localized_name":"Birmingham","country":"GB","state":"England","type":"Town","is_root":false,"urls":{"web":{"discover":"https://www.kickstarter.com/discover/places/birmingham-gb","location":"https://www.kickstarter.com/locations/birmingham-gb"},"api":{"nearby_projects":"https://api.kickstarter.com/v1/discover?signature=1560466559.ce78bdb2eed5b1dc5dfe2d35530ec37365a71825&woe_id=12723"}}}'

In [12]:
city = kick_inter['location'].str.partition("localized_name\":\"")[2]
kick_inter['city'] = city.str.partition("\"")[0]

In [13]:
country = kick_inter['location'].str.partition("country\":\"")[2]
kick_inter['country_loc'] = country.str.partition("\"")[0]

In [14]:
state_location = kick_inter['location'].str.partition("state\":\"")[2]
kick_inter['state_loc'] = state_location.str.partition("\"")[0]

In [23]:
kick_inter.drop(columns='location', inplace=True)

**LET'S BREAK UP THE CREATOR VARIABLE**

In [15]:
kick_inter.creator[3]

'{"id":467960938,"name":"Drank The Gold","slug":"drankthegold","is_registered":null,"chosen_currency":null,"is_superbacker":null,"avatar":{"thumb":"https://ksr-ugc.imgix.net/assets/025/633/415/617ba137f8b1831776af5ec109b58c01_original.jpg?ixlib=rb-2.1.0&w=40&h=40&fit=crop&v=1561660936&auto=format&frame=1&q=92&s=9068fbf7d0f507dcc5ba25c05a7751a3","small":"https://ksr-ugc.imgix.net/assets/025/633/415/617ba137f8b1831776af5ec109b58c01_original.jpg?ixlib=rb-2.1.0&w=160&h=160&fit=crop&v=1561660936&auto=format&frame=1&q=92&s=7517c86ac347cb90acd2883f6e38f4fa","medium":"https://ksr-ugc.imgix.net/assets/025/633/415/617ba137f8b1831776af5ec109b58c01_original.jpg?ixlib=rb-2.1.0&w=160&h=160&fit=crop&v=1561660936&auto=format&frame=1&q=92&s=7517c86ac347cb90acd2883f6e38f4fa"},"urls":{"web":{"user":"https://www.kickstarter.com/profile/drankthegold"},"api":{"user":"https://api.kickstarter.com/v1/users/467960938?signature=1563510715.2dd8dfc2be4143c67c8b43801d448300c4a0de4d"}}}'

In [16]:
creator_name = kick_inter['creator'].str.partition("name\":\"")[2]
kick_inter['creator_name'] = creator_name.str.partition("\"")[0]

In [17]:
creator_slug = kick_inter['creator'].str.partition("slug\":\"")[2]
kick_inter['creator_slug'] = creator_slug.str.partition("\"")[0]

In [24]:
kick_inter.drop(columns='creator', inplace=True)

**LET'S CONVERT ALL UNIX TIME COLUMNS TO DATETIME**

**UNIX TIME COLUMNS:** 
1. created_at
1. deadline
1. launched_at
1. state_changed_at

In [27]:
kick_inter.head()

,backers_count,blurb,converted_pledged_amount,country,created_at,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,last_update_published_at,launched_at,name,permissions,pledged,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,unread_messages_count,unseen_activity_count,urls,usd_pledged,usd_type,sub_category,overall_category,city,country_loc,state_loc,creator_name,creator_slug
0,0,"I'm just going to say it, I'm not special. I'm...",0.0,US,1563159576,USD,$,True,USD,1566018288,False,NaN,1.000000,5000.0,1893102245,NaN,True,NaN,NaN,1563426288,Shirt and hat,NaN,0.0,shirt-and-hat,https://www.kickstarter.com/discover/categorie...,False,False,live,1563426288,1.000000,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",0.0,domestic,Apparel,fashion/apparel,Wasilla,US,AK,Dima01,dima01
1,568,for Tabletop Role Playing Games like Dungeons ...,18969.0,US,1559509615,USD,$,True,USD,1563422100,False,NaN,1.000000,5000.0,1175125319,NaN,False,NaN,NaN,1560651641,RPG Minimalist Creature Dice & Status / Condit...,NaN,18969.0,rpg-minimalist-creature-dice-and-status-condit...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1563422101,1.000000,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",18969.0,domestic,Tabletop Games,games/tabletop games,Holland,US,MI,quEmpire Gaming,quempire
2,0,Giuliano Clothing is on a mission to reinvent ...,0.0,CA,1563405193,CAD,$,True,USD,1566013807,False,NaN,0.766388,5000.0,1290757180,NaN,True,NaN,NaN,1563421807,Giuliano Clothing: Modern Fashion,NaN,0.0,giuliano-clothing-modern-fashion,https://www.kickstarter.com/discover/categorie...,False,False,live,1563421807,0.766254,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",0.0,domestic,Fashion,fashion,Toronto,CA,ON,Giuliano Clothing,giulianoclothing
3,80,We have a new album that we are ready to relea...,3691.0,US,1561660600,USD,$,True,USD,1563420600,False,NaN,1.000000,3500.0,920424993,NaN,False,NaN,NaN,1561781837,Drank The Gold's new album: Sipped The Silver,NaN,3691.0,drank-the-golds-new-album-sipped-the-silver,https://www.kickstarter.com/discover/categorie...,True,True,successful,1563420600,1.000000,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",3691.0,domestic,Music,music,Saratoga Springs,US,NY,Drank The Gold,drankthegold
4,0,The film follows 4 frustrated campaigns as the...,0.0,US,1563392717,USD,$,True,USD,1566012181,False,NaN,1.000000,2000.0,255952264,NaN,True,NaN,NaN,1563420181,A Period Piece DVD Funding,NaN,0.0,a-period-piece-dvd-funding,https://www.kickstarter.com/discover/categorie...,False,False,live,1563420182,1.000000,NaN,NaN,"{""web"":{""project"":""https://www.kickstarter.com...",0.0,domestic,Drama,film & video/drama,Nashville,US,TN,Anthony Stephen Hamilton,aperiodpiece


In [29]:
kick_inter['created_at'] = pd.to_datetime(kick_inter['created_at'],unit='s')
kick_inter['deadline'] = pd.to_datetime(kick_inter['deadline'],unit='s')
kick_inter['launched_at'] = pd.to_datetime(kick_inter['launched_at'],unit='s')
kick_inter['state_changed_at'] = pd.to_datetime(kick_inter['state_changed_at'],unit='s')

In [30]:
kick_inter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332899 entries, 0 to 332898
Data columns (total 42 columns):
backers_count               332899 non-null int64
blurb                       332889 non-null object
converted_pledged_amount    195183 non-null float64
country                     332899 non-null object
created_at                  332899 non-null datetime64[ns]
currency                    332899 non-null object
currency_symbol             332899 non-null object
currency_trailing_code      332899 non-null bool
current_currency            195183 non-null object
deadline                    332899 non-null datetime64[ns]
disable_communication       332899 non-null bool
friends                     1629 non-null object
fx_rate                     185035 non-null float64
goal                        332899 non-null float64
id                          332899 non-null int64
is_backing                  1629 non-null object
is_starrable                206127 non-null object
is_starred   